In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from keras.metrics import Precision, Recall, AUC, CategoricalAccuracy

from custom_models.augmentation import basic_plus_color_augmentation, basic_augmentation
from custom_models.bilinear_cnns import fe_resnet
from custom_models.cnns import simple_cnn_bn, base_resnet50
from custom_models.optimization_utilities import get_standard_callbacks
from etl.load_dataset import DatasetProcessor, get_tf_eggim_patch_dataset
from optimization.custom_losses import weighted_categorical_crossentropy

2025-01-15 22:59:39.693055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736981979.715771   93410 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736981979.720188   93410 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 22:59:39.748274: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
os.getcwd()

'/home/goda/Desktop/tese/eggimazing'

In [3]:
target_dir = '../2025-01_EGGIM_Dataset3'  # aqui foi o o que eu chamei no remote server, mas fica a criterio
dp = DatasetProcessor(target_dir)
df = dp.process()

togas_ids_boolean = np.array([x.startswith('PT') for x in df['patient_id'].values])
df_togas = df[togas_ids_boolean].reset_index(drop=True)
df_ipo = df[~togas_ids_boolean].reset_index(drop=True)


In [27]:
def get_valid_patiend_ids(dataframe):
    df = dataframe.copy()
    p_ids = list(set(df['patient_id']))
    valid_patients = []
    for p_id in p_ids:
        p_lands = np.unique(df[df.patient_id == p_id].landmark).squeeze()
        p_lands = [x.split('.')[0] for x in p_lands]

        if 'ii' in p_lands or 'xii' in p_lands:
            if 'ix' in p_lands or 'x' in p_lands:
                if 'vi' in p_lands and\
                    'vii' in p_lands and\
                    'viii' in p_lands:
                    valid_patients.append(p_id)
    return valid_patients

valid_patients = get_valid_patiend_ids(df_togas)

In [29]:
np.save('configs/new_patient_ids.npy', valid_patients)

In [30]:
np.load('configs/new_patient_ids.npy', allow_pickle=True)

array(['PT053', 'PT016', 'PT049', 'PT074', 'PT045', 'PT081', 'PT004',
       'PT030', 'PT082', 'PT050', 'PT072', 'PT068', 'PT028', 'PT083',
       'PT037', 'PT084', 'PT070', 'PT034', 'PT060', 'PT015', 'PT067',
       'PT005', 'PT078', 'PT032', 'PT009', 'PT035', 'PT073', 'PT061',
       'PT051', 'PT041', 'PT075', 'PT013', 'PT077', 'PT063', 'PT039',
       'PT011', 'PT023', 'PT048', 'PT031', 'PT086', 'PT047', 'PT071',
       'PT040', 'PT076', 'PT064', 'PT029', 'PT066', 'PT021', 'PT054',
       'PT069', 'PT026', 'PT043', 'PT006', 'PT065', 'PT080', 'PT024',
       'PT025', 'PT038', 'PT022', 'PT062', 'PT079', 'PT020', 'PT036',
       'PT046', 'PT059'], dtype='<U5')

In [9]:
!pwd

/home/goda/Desktop/tese/eggimazing


In [11]:
debug_pids = ['PT004', 'PT005', 'PT006', 'PT009', 'PT016', 'PT020', 'PT021', 'PT022']


In [12]:
from etl.load_dataset import DatasetProcessor, get_tf_eggim_patch_dataset

In [13]:
df_togas

,bbox,eggim_global,eggim_square,landmark,image_directory,patient_id
0,"[218, 113, 442, 338]",0,0,ii. distal body in anteversion,../2025-01_EGGIM_Dataset3/20240117160000847.jpg,PT002
1,"[213, 228, 438, 452]",0,0,ix. distal lesser curvature in retroflexion,../2025-01_EGGIM_Dataset3/20240117160333928.jpg,PT002
2,"[190, 231, 415, 455]",0,0,ix. distal lesser curvature in retroflexion,../2025-01_EGGIM_Dataset3/20240117160406482.jpg,PT002
3,"[208, 235, 432, 460]",1,0,x. upper body in retroflexion,../2025-01_EGGIM_Dataset3/20240117160431270.jpg,PT002
4,"[314, 153, 539, 378]",0,0,vii. proximal antrum - greater curvature/poste...,../2025-01_EGGIM_Dataset3/20240119123159483.jpg,PT003
...,...,...,...,...,...,...
861,"[328, 212, 552, 436]",0,0,ix. distal lesser curvature in retroflexion,../2025-01_EGGIM_Dataset3/20241016182413017.jpg,PT086
862,"[323, 178, 547, 403]",0,0,x. upper body in retroflexion,../2025-01_EGGIM_Dataset3/20241016182430546.jpg,PT086
863,"[277, 140, 501, 364]",1,1,viii. incisura in retroflexion,../2025-01_EGGIM_Dataset3/20241016182711324.jpg,PT086
864,"[262, 161, 486, 386]",2,2,vii. proximal antrum - greater curvature/poste...,../2025-01_EGGIM_Dataset3/20241016182746880.jpg,PT086


In [14]:
import re
import json
import math
import pandas as pd
import json
import math
import numpy as np
import os
import pandas as pd
import re
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit, GroupShuffleSplit, LeavePGroupsOut
from sklearn.utils import resample


class DatasetProcessor:
    def __init__(self, target_directory, image_extensions=('jpg'), annotation_extensions=('json'), id_prefix_size=10):
        self.target_directory = target_directory
        self.image_extensions = image_extensions
        self.annotation_extension = annotation_extensions
        # We assume that the patient id is encoded in the first id_prefix_size numbers of each file
        self.id_prefix_size = id_prefix_size
        self.dataset_dictionary = self._load_file_names()

    def _load_file_names(self):
        dataset_files = sorted(os.listdir(self.target_directory))
        json_names = [x for x in dataset_files if x.endswith('.json')]
        image_names = [x for x in dataset_files if x not in json_names]
        patient_ids = np.array([x[:self.id_prefix_size] for x in json_names])
        return {pid_: [[image_ for image_ in image_names if image_.startswith(pid_)],
                       [json_ for json_ in json_names if json_.startswith(pid_)]] for pid_ in patient_ids}

    # bbox, eggim in bbox, landmark
    def process_json(self, directory):
        with open(directory, 'r') as file:
            data = json.load(file)
        dict_parameters = {}
        for instance in data['instances']:
            if instance['className'] == 'EGGIM in the FULL Anatomical Location':
                dict_parameters['eggim_global'] = int(instance['attributes'][0]['name'])
            if instance['className'] == 'EGGIM in Target Area - Square':
                dict_parameters['eggim_square'] = int(instance['attributes'][0]['name'])
            if instance['className'] == 'Anatomical Location':
                dict_parameters['landmark'] = str(instance['attributes'][0]['name'])
            if instance['className'] == 'Comments':
                if not instance['attributes']:  # check if list is empty
                    continue
                else:
                    id_ = str(instance['attributes'][0]['name'])
                    id_ = re.split(r'[ \n]+', id_)[0]
                    if id_.startswith('PT'):  # This is necessary to mark the patients from togas
                        dict_parameters['patient_id'] = id_
            if instance["type"] == "bbox" and "points" in instance:
                points = instance["points"]
                left = points["x1"]
                top = points["y1"]
                right = points["x2"]
                bottom = points["y2"]
                # print("x1", left, "y1", top, "x2", right, "y2", bottom)
                dict_parameters['bbox'] = np.array([math.floor(left), math.floor(top), math.floor(right), math.floor(
                    bottom)])  # plt.imshow(np.array(image)[round(y1):round(y2), round(x1):round(x2), :])
        return dict_parameters

    def process(self, merge_eggim_square=False, merge_eggim_global=False):
        dataset_info = []
        for patient_id, (images, jsons) in self.dataset_dictionary.items():
            for x, y in zip(images, jsons):
                annotation_data = self.process_json(os.path.join(self.target_directory, y))
                annotation_data['image_directory'] = os.path.join(self.target_directory, x)
                if 'patient_id' not in annotation_data:
                    annotation_data['patient_id'] = patient_id
                dataset_info.append(annotation_data)
        df = pd.DataFrame(dataset_info)
        if merge_eggim_square:
            df['eggim_square'] = df['eggim_square'].apply(lambda score: 0 if score == 0 else 1)
        if merge_eggim_global:
            df['eggim_global'] = df['eggim_global'].apply(lambda score: 0 if score == 0 else 1)
        return df

    @staticmethod
    def patient_wise_split(df_target,
                           df_extra,
                           patients_ids,
                           internal_train_size=0.5,
                           target_variable='eggim_square',
                           random_state=None):

        assert (0 < internal_train_size) and (internal_train_size < 1)
        for patient_id in patients_ids:
            test_frames_rows = df_target['patient_id'] == patient_id

            df_test = df_target.loc[test_frames_rows]
            df_temp = df_target.loc[~test_frames_rows]
            X_temp = df_temp.drop(columns=[target_variable])
            y_temp = df_temp[target_variable]
            sss_temp = StratifiedShuffleSplit(n_splits=1, train_size=internal_train_size,
                                              test_size=1. - internal_train_size, random_state=random_state)
            train_idx, val_idx = next(sss_temp.split(X_temp, y_temp))

            df_train = df_target.iloc[train_idx]
            df_val = df_target.iloc[val_idx]
            df_train = DatasetProcessor.augment_dataframe_stratified(df_train,
                                                                     df_extra,
                                                                     target_column=target_variable)
            df_train = pd.concat([df_train, df_extra], axis=0)

            yield df_train, df_val, df_test

In [15]:
import numpy as np
target_dir = '../test_files/EGGIMazing/Dataset01'
patient_ids = np.load('../configs/test_patient_ids_2.npy', allow_pickle=True)
dp = DatasetProcessor(target_dir)
df = dp.process()

togas_ids_boolean = np.array([x.startswith('PT') for x in df['patient_id'].values])
df_togas = df[togas_ids_boolean].reset_index(drop=True)
df_ipo = df[~togas_ids_boolean].reset_index(drop=True)

split = dp.patient_wise_split(df_togas,
                              df_ipo,
                              patient_ids,
                              internal_train_size=0.9,
                              target_variable='eggim_square',
                              random_state=42)

FileNotFoundError: [Errno 2] No such file or directory: '../configs/test_patient_ids_2.npy'

In [14]:
bboxes = np.stack(np.array(df_togas['bbox'].values), axis=-1).T

In [15]:
pt04 = df_togas[df_togas['patient_id'] == 'PT004']
pt24 = df_togas[df_togas['patient_id'] == 'PT024']

In [16]:
np.unique(df_togas.patient_id)

array(['PT002', 'PT003', 'PT004', 'PT005', 'PT006', 'PT007', 'PT008',
       'PT009', 'PT010', 'PT011', 'PT012', 'PT013', 'PT014', 'PT015',
       'PT016', 'PT017', 'PT018', 'PT019', 'PT020', 'PT021', 'PT022',
       'PT023', 'PT024', 'PT025', 'PT026', 'PT027', 'PT028', 'PT029',
       'PT030', 'PT031', 'PT032', 'PT034', 'PT035', 'PT036', 'PT037',
       'PT038', 'PT039', 'PT040', 'PT041', 'PT042', 'PT043', 'PT044',
       'PT045', 'PT046', 'PT047', 'PT048', 'PT049', 'PT050', 'PT051',
       'PT052', 'PT053'], dtype=object)

In [12]:
from etl.load_dataset import get_tf_eggim_patch_dataset
import tensorflow as tf
tf_test_df = get_tf_eggim_patch_dataset(pt24, num_classes=3,
                                                preprocess_fn=tf.keras.applications.resnet.preprocess_input)

(12, 4)


2024-11-08 16:51:36.049270: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 16:51:36.049526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 16:51:36.096861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [65]:
patient_ids = np.load('../configs/test_patient_ids_2.npy', allow_pickle=True)
split = dp.patient_wise_split(df_togas,
                                  df_ipo,
                                  patient_ids,
                                  internal_train_size=0.9,
                                  target_variable='eggim_square',
                                  random_state=42)

In [62]:
for fold, (df_train, df_val, df_test) in enumerate(split):
    if patient_ids[fold] == 'PT004':
        pt04 = df_test
    if patient_ids[fold] == 'PT024':

In [76]:
np.load('../configs/test_patient_ids.npy', allow_pickle=True).shape

(8,)

In [83]:
np.load('../configs/test_patient_ids.npy', allow_pickle=True), np.load('../configs/test_patient_ids_2.npy', allow_pickle=True)

(array(['PT004', 'PT005', 'PT006', 'PT009', 'PT016', 'PT020', 'PT021',
        'PT022'], dtype=object),
 array(['PT004', 'PT005', 'PT006', 'PT009', 'PT016', 'PT020', 'PT021',
        'PT022', 'PT024', 'PT025', 'PT028', 'PT029', 'PT030', 'PT031',
        'PT034', 'PT035', 'PT037', 'PT039', 'PT040', 'PT041', 'PT045',
        'PT046', 'PT048', 'PT049', 'PT051', 'PT053'], dtype=object))